-------
## Prédiction de la Polarité - MLP
-------

In [ ]:
import pandas as pd
import pickle
import tensorflow as tf
import re
from sklearn.metrics import classification_report, accuracy_score, f1_score, log_loss

model = tf.keras.models.load_model('model_polarite.keras')

with open('tfidf_polarite.pkl', 'rb') as f:
    tfidf = pickle.load(f)
    
with open('label_encoder_polarite.pkl', 'rb') as f:
    le = pickle.load(f)

def clean_text(text):
    """Nettoyage du texte selon votre Cellule 5"""
    text = str(text).lower()
    text = re.sub(r"http\S+", " ", text)       # enlever liens
    text = re.sub(r"[^a-z\s']", " ", text)     # garder lettres et apostrophes
    text = re.sub(r"\s+", " ", text).strip()   # espaces propres
    return text

def label_polarity(score):
    """Conversion des étoiles en classes selon votre Cellule 5"""
    if score > 3: return "positif"
    elif score < 3: return "négatif"
    else: return "neutre"

file_prof = r'C:\Users\lucie\OneDrive\Documents\SEMESTRE_6\test\Test_file_SAE.json'
df_prof = pd.read_json(file_prof, lines=True)

texts_cleaned = df_prof['text'].apply(clean_text)
# Vectorisation TF-IDF (conversion en matrice dense pour le MLP)
X_prof_tfidf = tfidf.transform(texts_cleaned).toarray()

df_prof['label_reel'] = df_prof['stars'].apply(label_polarity)
y_true_indices = le.transform(df_prof['label_reel'])

# Prédictions
predictions_probs = model.predict(X_prof_tfidf)
predictions_indices = predictions_probs.argmax(axis=1)

df_prof['prediction'] = le.inverse_transform(predictions_indices)

acc = accuracy_score(y_true_indices, predictions_indices)
f1_weighted = f1_score(y_true_indices, predictions_indices, average='weighted')
loss = log_loss(y_true_indices, predictions_probs)

target_names = [str(cls) for cls in le.classes_]

print("\n" + "="*60)
print(f"RÉSULTATS DE PERFORMANCE (Groupe 3 - MLP)")
print(f"Précision (Accuracy) : {acc:.4f}")
print(f"F1-Score (pondéré)    : {f1_weighted:.4f}")
print(f"Perte (Log Loss)      : {loss:.4f}")
print("\n" + "-"*60)
print("Rapport détaillé :\n")
print(classification_report(y_true_indices, predictions_indices, target_names=target_names))
print("\n" + "="*60)

# Sauvegarde finale avec les colonnes d'origine et la prédiction
df_prof.to_csv('Resultat_POLARITE_Groupe3_MLP.csv', index=False)
print("Fichier 'Resultat_POLARITE_Groupe3_MLP.csv' généré avec succès.")

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

RÉSULTATS DE PERFORMANCE (Groupe 3 - MLP)
Précision (Accuracy) : 0.8202
F1-Score (pondéré)    : 0.8378
Perte (Log Loss)      : 0.4471

------------------------------------------------------------
Rapport détaillé :

              precision    recall  f1-score   support

      neutre       0.33      0.57      0.42       488
     négatif       0.83      0.85      0.84      1131
     positif       0.96      0.85      0.90      3381

    accuracy                           0.82      5000
   macro avg       0.70      0.76      0.72      5000
weighted avg       0.87      0.82      0.84      5000


Fichier 'Resultat_POLARITE_Groupe3_MLP.csv' généré avec succès.


-------
## Prédiction score (étoiles) - MLP
-------

In [ ]:
import pandas as pd
import pickle
import tensorflow as tf
import re
from sklearn.metrics import classification_report, accuracy_score, f1_score, log_loss

model = tf.keras.models.load_model('model_etoiles.keras')
with open('tfidf_etoiles.pkl', 'rb') as f:
    tfidf = pickle.load(f)
    
with open('label_encoder_etoiles.pkl', 'rb') as f:
    le = pickle.load(f)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", " ", text)       # enlever les liens
    text = re.sub(r"[^a-z\s']", " ", text)     # garder lettres et apostrophes
    text = re.sub(r"\s+", " ", text).strip()   # espaces propres
    return text

file_prof = r'C:\Users\lucie\OneDrive\Documents\SEMESTRE_6\test\Test_file_SAE.json'
df_prof = pd.read_json(file_prof, lines=True)

texts_cleaned = df_prof['text'].apply(clean_text)

X_prof_tfidf = tfidf.transform(texts_cleaned).toarray()
y_true_indices = le.transform(df_prof['stars'])

# Prédiction des probabilités
predictions_probs = model.predict(X_prof_tfidf)
predictions_indices = predictions_probs.argmax(axis=1)
df_prof['prediction_etoile'] = le.inverse_transform(predictions_indices)

acc = accuracy_score(y_true_indices, predictions_indices)
f1_weighted = f1_score(y_true_indices, predictions_indices, average='weighted')
loss = log_loss(y_true_indices, predictions_probs)

target_names = [f"{cls} étoile(s)" for cls in le.classes_]

print("\n" + "="*60)
print(f"RÉSULTATS DE PERFORMANCE - MODÈLE ÉTOILES")
print(f"Précision (Accuracy) : {acc:.4f}")
print(f"F1-Score (pondéré)    : {f1_weighted:.4f}")
print(f"Perte (Log Loss)      : {loss:.4f}")
print("\n" + "-"*60)
print("Rapport détaillé par nombre d'étoiles :\n")
print(classification_report(y_true_indices, predictions_indices, target_names=target_names))
print("="*60)

# Sauvegarde du fichier final
df_prof.to_csv('Resultat_ETOILES_Groupe3_MLP.csv', index=False)
print("Fichier 'Resultat_ETOILES_Groupe3_MLP.csv' généré avec succès.")

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

RÉSULTATS DE PERFORMANCE - MODÈLE ÉTOILES
Précision (Accuracy) : 0.6094
F1-Score (pondéré)    : 0.6213
Perte (Log Loss)      : 0.8912

------------------------------------------------------------
Rapport détaillé par nombre d'étoiles :

              precision    recall  f1-score   support

 1 étoile(s)       0.77      0.71      0.74       747
 2 étoile(s)       0.35      0.41      0.38       384
 3 étoile(s)       0.42      0.41      0.41       488
 4 étoile(s)       0.43      0.59      0.49      1096
 5 étoile(s)       0.81      0.66      0.73      2285

    accuracy                           0.61      5000
   macro avg       0.55      0.56      0.55      5000
weighted avg       0.65      0.61      0.62      5000

Fichier 'Resultat_ETOILES_Groupe3_MLP.csv' généré avec succès.
